# 从训练到部署一键式烟头检测

环境问题是当前社会关注的重点，部分烟民乱丢烟头不仅让城市环境“大打折扣”，还极易引起安全隐患。
* 一是乱扔烟头极易造成火灾。烟头虽小，但其表面温度一般在200℃～300℃之间，中心温度可达700℃～800℃之间，而一般可燃物（如纸张、棉花、柴草、木材等）的燃点都在130℃～350℃，都低于烟头的温度。而且如果香烟明火遇见化学物质，就会导致爆炸。
* 二是乱扔烟头对植被生长造成不良影响。经过有关部门调查研究，烟头会对青草的发芽率和枝条长度造成影响，对植被生长有负面作用。而且烟头主要成分是醋酸纤维素，这种物质在自然界分解需要10年。
* 三是烟头对城市卫生造成污染。现在人民生活水平提高，城市卫生也提上了日程。很多城市越来越讲究卫生，而乱扔烟头，不仅影响城市的美观，也仿佛成了路面上的牛皮癣，不仅到处都是，还如杂草一般，野火烧不尽，春风吹又生。

本项目基于PP-YOLOE-SOD模型设计了一个烟头检测算法，完善了从训练到部署的全流程实践，在验证集上能够达到92.0%的mAP，并且能够通过FastDeploy开发套件快速部署，能够满足我们实际检测的需求。

## 一、数据集简介

本项目使用的是roboflow制作的烟头检测数据集Cigarrette_Detection Computer Vision Project，总共提供1023张标注好的数据图像，该数据集只有一个标签Bud。该数据集已经包含部分数据增强操作：50%可能性的水平翻转、50%可能性的竖直翻转、90度旋转、0-0.25像素的高斯滤波和1%像素点的椒盐噪声。该数据集现已上传至AI Studio。
数据集链接：
* [原链接](https://universe.roboflow.com/cigarette/cigarrette_detection)
* [VOC](https://aistudio.baidu.com/aistudio/datasetdetail/198353)
* [COCO](https://aistudio.baidu.com/aistudio/datasetdetail/198347)

部分训练集图片：

![](https://ai-studio-static-online.cdn.bcebos.com/6d0d1b549c524a8896d6255b467492c16f69fec397934a35bf7052e74fe8a795)

## 二、数据预处理

**Step01：** 解压数据集

In [ ]:
!unzip /home/aistudio/data/data198353/Cigarrette_Detection.v30i.voc.zip -d /home/aistudio/work/

**Step02：** 区分文件夹中不同后缀名的文件

工作目录：/home/aistudio/work/

本项目用到的是train、vaild、test文件夹中的图片和标注数据，由于图片和标注数据是存放在一起的，所以我们首先需要把两者分开存放，方便后续处理。

首先，我们分别在train、valid和test目录下新建两个文件夹分别为JPEGImages和Annotations。

JPEGImages用于存放数据集中的图片。

Annotations用于存放标注文件。

然后通过下面的指令移动相同后缀名的文件到指定文件夹。

In [ ]:
%cd /home/aistudio/work/train/
!mkdir JPEGImages 
!mkdir Annotations
!mv /home/aistudio/work/train/*.jpg /home/aistudio/work/train/JPEGImages/
!mv /home/aistudio/work/train/*.xml /home/aistudio/work/train/Annotations/

In [ ]:
%cd /home/aistudio/work/valid/
!mkdir JPEGImages 
!mkdir Annotations
!mv /home/aistudio/work/valid/*.jpg /home/aistudio/work/valid/JPEGImages/
!mv /home/aistudio/work/valid/*.xml /home/aistudio/work/valid/Annotations/

In [ ]:
%cd /home/aistudio/work/test/
!mkdir JPEGImages 
!mkdir Annotations
!mv /home/aistudio/work/test/*.jpg /home/aistudio/work/test/JPEGImages/
!mv /home/aistudio/work/test/*.xml /home/aistudio/work/test/Annotations/

**Step03：** 生成数据集txt文件

In [20]:
import os
import random

# 设置图片路径和txt存放的路径
train_file_path = '/home/aistudio/work/train/JPEGImages'
train_save_Path = '/home/aistudio/work/train/'
valid_file_path = '/home/aistudio/work/valid/JPEGImages'
valid_save_Path = '/home/aistudio/work/valid/'

total_train_image = os.listdir(train_file_path)                 
ftrain = open(os.path.join(train_save_Path, 'train.txt'), 'w')
total_valid_image = os.listdir(valid_file_path)                 
fvalid = open(os.path.join(valid_save_Path, 'val.txt'), 'w')

# 进行写文件名
for file in total_train_image:
    name = file[:-4]  # 去掉拓展名
    ftrain.write("JPEGImages/"+name+".jpg "+"Annotations/"+name+".xml"+"\n")
for file in total_valid_image:
    name = file[:-4]  # 去掉拓展名
    fvalid.write("JPEGImages/"+name+".jpg "+"Annotations/"+name+".xml"+"\n")

# 关闭txt文件
ftrain.close()
fvalid.close()

我们可以看到运行以上代码块后就生成了数据集的txt文件，但还缺少标签文件，因为该任务是个单目标检测问题，只有一个类别，那我们不妨手动创建个label.txt，里面存放唯一类别Bud。

## 三、代码实现

### 3.1 检测数据分析

通过运行以下代码块，我们可以看到数据集有且仅有一个标签Bud，验证了我们上面生成的标签文件是没有什么问题的。

In [2]:
import os
from unicodedata import name
import xml.etree.ElementTree as ET
import glob

def count_num(indir):
    # 提取xml文件列表
    os.chdir(indir)
    annotations = os.listdir('.')
    annotations = glob.glob(str(annotations) + '*.xml')

    dict = {} # 新建字典，用于存放各类标签名及其对应的数目
    for i, file in enumerate(annotations): # 遍历xml文件
       
        # actual parsing
        in_file = open(file, encoding = 'utf-8')
        tree = ET.parse(in_file)
        root = tree.getroot()

        # 遍历文件的所有标签
        for obj in root.iter('object'):
            name = obj.find('name').text
            if(name in dict.keys()): dict[name] += 1 # 如果标签不是第一次出现，则+1
            else: dict[name] = 1 # 如果标签是第一次出现，则将该标签名对应的value初始化为1

    # 打印结果
    print("各类标签的数量分别为：")
    for key in dict.keys(): 
        print(key + ': ' + str(dict[key]))            

indir='/home/aistudio/work/train/Annotations/'   # xml文件所在的目录
count_num(indir) # 调用函数统计各类标签数目

各类标签的数量分别为：
Bud: 928


**检测框高宽比分析：** 通过绘制检测框高宽比分布直方图反映当前检测框宽高比的分布情况。

In [ ]:
import os
from unicodedata import name
import xml.etree.ElementTree as ET
import glob
import matplotlib.pyplot as plt

def ratio(indir):
    # 提取xml文件列表
    os.chdir(indir)
    annotations = os.listdir('.')
    annotations = glob.glob(str(annotations) + '*.xml')
    # count_0, count_1, count_2, count_3 = 0, 0, 0, 0 # 举反例，不要这么写
    count = [0 for i in range(20)]

    for i, file in enumerate(annotations): # 遍历xml文件
        # actual parsing
        in_file = open(file, encoding = 'utf-8')
        tree = ET.parse(in_file)
        root = tree.getroot()

        # 遍历文件的所有检测框
        for obj in root.iter('object'):
            xmin = obj.find('bndbox').find('xmin').text
            ymin = obj.find('bndbox').find('ymin').text
            xmax = obj.find('bndbox').find('xmax').text
            ymax = obj.find('bndbox').find('ymax').text
            Aspect_ratio = (int(ymax)-int(ymin)) / (int(xmax)-int(xmin))
            if int(Aspect_ratio/0.25) < 19:
                count[int(Aspect_ratio/0.25)] += 1
            else:
                count[-1] += 1
    sign = [0.25*i for i in range(20)]
    plt.bar(x=sign, height=count)
    plt.savefig("/home/aistudio/work/hw.png") 
    plt.show()
    print(count)

indir='/home/aistudio/work/train/Annotations'   # xml文件所在的目录
ratio(indir)

结果如下：

![](https://ai-studio-static-online.cdn.bcebos.com/3845ff6b2df24ff5a055de53561893f79d6d8f943cc6460ba0b5e1c72b1c3546)

**图像尺寸分析：** 通过图像尺寸分析，我们可以看到该数据集图片共有三种尺寸，分别是：
* [2048, 1152]
* [1536, 2048]
* [2048, 1536]

In [ ]:
import os
from unicodedata import name
import xml.etree.ElementTree as ET
import glob

def Image_size(indir):
    # 提取xml文件列表
    os.chdir(indir)
    annotations = os.listdir('.')
    annotations = glob.glob(str(annotations) + '*.xml')
    width_heights = []

    for i, file in enumerate(annotations): # 遍历xml文件
        # actual parsing
        in_file = open(file, encoding = 'utf-8')
        tree = ET.parse(in_file)
        root = tree.getroot()
        width = int(root.find('size').find('width').text)
        height = int(root.find('size').find('height').text)
        if [width, height] not in width_heights: width_heights.append([width, height])
    print("数据集中，有{}种不同的尺寸，分别是：".format(len(width_heights)))
    for item in width_heights:
        print(item)

indir='/home/aistudio/work/train/Annotations/'   # xml文件所在的目录
Image_size(indir)

### 3.2 安装PaddleDetection

In [ ]:
# 克隆PaddleDetection仓库
#!git clone https://github.com/PaddlePaddle/PaddleDetection.git

# 安装其他依赖
%cd /home/aistudio/PaddleDetection/
!pip install -r requirements.txt

# 编译安装paddledet
!python setup.py install

### 3.3 模型训练

**Step01：** 在/home/aistudio/PaddleDetection/dataset目录下新建一个名为Cigarrette_Detection的文件夹，并将数据集移动到该目录下。

In [5]:
!mv /home/aistudio/work/* /home/aistudio/PaddleDetection/dataset/Cigarrette_Detection

Step02：统计数据集分布

在本项目任务是要检测烟头，烟头在图像中是一个很小的物体，因此我想到使用PP-YOLOE-SOD模型。

PP-YOLOE-SOD 是PaddleDetection团队自研的小目标检测特色模型，使用数据集分布相关的基于向量的DFL算法 和 针对小目标优化的中心先验优化策略，并且在模型的Neck(FPN)结构中加入Transformer模块，以及结合增加P2层、使用large size等策略，最终在多个小目标数据集上达到极高的精度。

在此之前，我首先需要统计一下数据集分布，看是否需要切图训练。

由于，官方给出的API目前只支持COCO格式数据集，所以我通过以下命令将数据集转化成COCO格式。

In [ ]:
!python tools/x2coco.py \
        --dataset_type voc \
        --voc_anno_dir dataset/Cigarrette_Detection/train/ \
        --voc_anno_list dataset/Cigarrette_Detection/train/train.txt \
        --voc_label_list dataset/Cigarrette_Detection/train/label.txt \
        --voc_out_name dataset/Cigarrette_Detection/train/voc_train.json

In [ ]:
!python tools/x2coco.py \
        --dataset_type voc \
        --voc_anno_dir dataset/Cigarrette_Detection/valid/ \
        --voc_anno_list dataset/Cigarrette_Detection/valid/val.txt \
        --voc_label_list dataset/Cigarrette_Detection/valid/label.txt \
        --voc_out_name dataset/Cigarrette_Detection/valid/voc_valid.json

In [ ]:
!python tools/box_distribution.py --json_path dataset/Cigarrette_Detection/train/voc_train.json --out_img box_distribution.jpg --eval_size 640 --small_stride 8

结果如下：
* Suggested reg_range[1] is 16 # DFL算法中推荐值，在 PP-YOLOE-SOD 模型的配置文件的head中设置为此值，效果最佳
* Mean of all img_w is 1913.986577181208 # 原图宽的平均值
* Mean of all img_h is 1390.3892617449665 # 原图高的平均值
* Median of ratio_w is 0.10880533854166666 # 标注框的宽与原图宽的比例的中位数
* Median of ratio_h is 0.14029947916666669 # 标注框的高与原图高的比例的中位数
* all_img with box:  894 # 数据集图片总数(排除无框或空标注的图片)
* all_ann:  928 # 数据集标注框总数

![](https://ai-studio-static-online.cdn.bcebos.com/330e8df0aab54dac943589733d3eb8b8cc5e5cd362814a93b74bab22dcf6ad71)

一般情况下，在原始数据集全部有标注框的图片中，原图宽高的平均值大于1500像素，且有1/2以上的图片标注框的平均宽高与原图宽高比例小于0.04时(通过打印中位数得到该值)，建议进行切图训练。因此在本项目中我选择通过原图去训练。

经过八十轮次的迭代，我们可以看到训练的模型已经在验证集取得了不错的效果，mAP为92.0%，满足了我们项目的标准。

In [ ]:
!python tools/train.py -c configs/smalldet/ppyoloe_plus_sod_crn_l_80e_coco.yml --amp --eval --use_vdl True --vdl_log_dir vdl_log_dir/scalar

损失函数如图所示：

![](https://ai-studio-static-online.cdn.bcebos.com/8c8d94cc5ef6456fa1c61715c2bac61cb7de03d3f01642b7b94e2132385e1208)

### 3.4 模型评估

通过如下命令在单个GPU上评估我们的验证集。

In [ ]:
!python tools/eval.py -c configs/smalldet/ppyoloe_plus_sod_crn_l_80e_coco.yml -o weights=output/ppyoloe_plus_sod_crn_l_80e_coco/best_model.pdparams

结果如下：
* Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.920
* Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 1.000
* Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.996
* Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
* Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
* Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.920
* Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.906
* Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.944
* Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.944
* Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
* Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
* Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.944
* Total sample number: 64, average FPS: 15.98533735534183

### 3.5 模型推理

我们可以通过以下命令在单张GPU上推理文件中的所有图片。

In [ ]:
!python tools/infer.py -c configs/smalldet/ppyoloe_plus_sod_crn_l_80e_coco.yml -o weights=output/ppyoloe_plus_sod_crn_l_80e_coco/best_model.pdparams --infer_dir=dataset/Cigarrette_Detection/test/JPEGImages --output_dir infer_output/

### 3.6 模型导出

PP-YOLO-SOD在GPU上部署或者速度测试需要通过tools/export_model.py导出模型。

In [ ]:
!python tools/export_model.py -c configs/smalldet/ppyoloe_plus_sod_crn_l_80e_coco.yml -o weights=output/ppyoloe_plus_sod_crn_l_80e_coco/best_model.pdparams

### 3.7 FastDeploy快速部署

**环境准备：** 本项目的部署环节主要用到的套件为飞桨部署工具FastDeploy，因此我们先安装FastDeploy。

In [ ]:
!pip install fastdeploy-gpu-python -f https://www.paddlepaddle.org.cn/whl/fastdeploy.html

**部署模型：**

导入飞桨部署工具FastDepoy包，创建Runtimeoption，具体实现如下代码所示。

In [ ]:
import fastdeploy as fd
import cv2
import os

In [ ]:
def build_option(device='cpu', use_trt=False):
    option = fd.RuntimeOption()

    if device.lower() == "gpu":
        option.use_gpu()

    if use_trt:
        option.use_trt_backend()
        option.set_trt_input_shape("image", [1, 3, 640, 640])
        option.set_trt_input_shape("scale_factor", [1, 2])

    return option

配置模型路径，创建Runtimeoption，指定部署设备和后端推理引擎，代码实现如下所示。

In [13]:
# 配置模型路径
model_path = '/home/aistudio/PaddleDetection/output_inference/ppyoloe_plus_sod_crn_l_80e_coco'
image_path = '/home/aistudio/PaddleDetection/dataset/Cigarrette_Detection/test/JPEGImages/ca1349ee-fbd8-4939-90ab-0118f04e602c_png.rf.72c05d0cb16fd458f1587cca162df699.jpg'
model_file = os.path.join(model_path, "model.pdmodel")
params_file = os.path.join(model_path, "model.pdiparams")
config_file = os.path.join(model_path, "infer_cfg.yml")

# 创建RuntimeOption
runtime_option = build_option(device='gpu', use_trt=False)

# 创建PPYOLOE模型
model = fd.vision.detection.PPYOLOE(model_file,
                                   params_file,
                                   config_file,
                                   runtime_option=runtime_option)

# 预测图片检测结果
im = cv2.imread(image_path)
result = model.predict(im.copy())
print(result)

# 预测结果可视化
vis_im = fd.vision.vis_detection(im, result, score_threshold=0.5, line_size=5, font_size=1.0)
cv2.imwrite("/home/aistudio/work/visualized_result.jpg", vis_im)
print("Visualized result save in ./visualized_result.jpg")

[INFO] fastdeploy/vision/common/processors/transform.cc(45)::FuseNormalizeCast	Normalize and Cast are fused to Normalize in preprocessing pipeline.
[INFO] fastdeploy/vision/common/processors/transform.cc(93)::FuseNormalizeHWC2CHW	Normalize and HWC2CHW are fused to NormalizeAndPermute  in preprocessing pipeline.
[INFO] fastdeploy/vision/common/processors/transform.cc(159)::FuseNormalizeColorConvert	BGR2RGB and NormalizeAndPermute are fused to NormalizeAndPermute with swap_rb=1
[INFO] fastdeploy/runtime/runtime.cc(336)::CreateOrtBackend	Runtime initialized with Backend::ORT in Device::GPU.
DetectionResult: [xmin, ymin, xmax, ymax, score, label_id]
699.563171,810.522095, 799.750000, 1026.120483, 0.947899, 0
413.548218,2002.615234, 463.452667, 2045.350220, 0.057318, 0
0.069912,2014.856812, 61.427635, 2043.998413, 0.047085, 0
-5.364040,2000.043213, 69.566093, 2050.473877, 0.025710, 0
324.293274,1981.731201, 383.768951, 2044.225952, 0.025672, 0
707.287048,2005.951416, 782.452332, 2046.240601

推理结果如下：

![](https://ai-studio-static-online.cdn.bcebos.com/d4d7e63542444a56be9bb02bb57a71073f28f3a0cb6749969e096a9d44450f1f)

## 四、效果展示

部分可视化结果如下：

![](https://ai-studio-static-online.cdn.bcebos.com/4ef85110be7c4280ab51da6b5269a6cbce082c493876456784f5289f8aa0af31)

## 五、总结提高

本项目任务要检测烟头，由于烟头目标较小，我们可以优先考虑小目标检测模型，因此我用的是PP-YOLOE-SOD，最终也取到了不错的效果，mAP达到92.0%，大家如果感兴趣可以选择通用目标检测模型，看看是否能达到更好的效果。同时我们的数据集已经做了部分数据增强的操作，因此不需要过多的数据增强，我在这里只多做了一个MixUp的操作。

作者简介：Submerge. 江苏某大学大三学生 人工智能专业 [主页链接](https://aistudio.baidu.com/aistudio/personalcenter/thirdview/2365489) 欢迎互关！

飞桨导师：刘建建 [Javaroom](https://aistudio.baidu.com/aistudio/personalcenter/thirdview/89263) 在此感谢。